**Computing Stress Residuals by taking spatial derivatives**
-------------------------------------------------------------------
1. Interpolate the three layers to a common grid
2. Compute spatial deriavtives and form divergence of stress field (1-point numerical difference scheme)
3. Find the smallest stress increment in each voxel that will bring balance of forces

In [ ]:
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.ndimage

In [ ]:
keys = ['layer_top', 'layer_center', 'layer_bottom']
dirs = [os.path.join('..', dir, 'test_run') for dir in keys]
sample = {}
sample['layer_top']    = np.fliplr(  np.load(os.path.join(dirs[0], 'sample_mask_layer_top.npy'))[5:-5, 5:-5]  )
sample['layer_center'] = np.fliplr(  np.load(os.path.join(dirs[1], 'sample_mask_layer_center.npy'))[2:-2, 2:-2] )
sample['layer_bottom'] = np.fliplr(  np.load(os.path.join(dirs[2], 'sample_mask_layer_bottom.npy'))[2:-2, 2:-2] )

stress = {}
stress['layer_top'] = np.load('../layer_top/test_run/stress_map_top_layer_refined_com_corrected.npy')
stress['layer_center'] = np.load('../layer_center/test_run/stress_map_central_layer_refined_com_corrected.npy')
stress['layer_bottom'] = np.load('../layer_bottom/test_run/stress_map_bottom_layer_refined_com_corrected.npy')

X, Y = {},{}
for i,key in enumerate(keys):
    X[key] = np.load( os.path.join( dirs[i], 'X_coord_gmap.npy' ) )
    Y[key] = np.load( os.path.join( dirs[i], 'Y_coord_gmap.npy' ) )

grain_boundaries = {}
for i,key in enumerate(keys):
    grain_boundaries[key] = np.load(key + '_grain_boundaries.npy')


In [ ]:
from scipy.interpolate import RegularGridInterpolator
from scipy.ndimage import median_filter
Xr, Yr = X[keys[1]], Y[keys[1]]
m,n,_,_ = stress[keys[1]].shape
stress_3D_interp = np.zeros((3,m,n,3,3))
for zi,key in enumerate(keys):
    for i in range(3):
        for j in range(3):
            interp = RegularGridInterpolator((X[key][:,0], Y[key][0,:]), stress[key][:,:,i,j], bounds_error=False, fill_value=np.nan, method='linear' )
            stress_3D_interp[zi,:,:, i, j] = interp((Xr, Yr))
stress_mask = sample[keys[1]].copy()

# interpolate in z to make the voxel dimensions equal dx=dy=dz
stress_3D_interp[0] = (stress_3D_interp[0] + stress_3D_interp[1])/2.
stress_3D_interp[2] = (stress_3D_interp[1] + stress_3D_interp[2])/2.

stress_3D_interp[:,~stress_mask,:,:] = np.nan

In [ ]:
d = [[None,None,None],[None,None,None],[None,None,None]]

for i in range(3):
    for j in range(3):

        if i>j:
            d[i][j] = d[j][i]
        else:
            m,n,o,_,_ = stress_3D_interp.shape

            # one-point gradient operator.
            gz = np.diff(  stress_3D_interp[:,:,:,i,j], axis=0, append=np.zeros((1,n,o)) )
            gx = np.diff(  stress_3D_interp[:,:,:,i,j], axis=1, append=np.zeros((m,1,o)) )
            gy = np.diff(  stress_3D_interp[:,:,:,i,j], axis=2, append=np.zeros((m,n,1)) )

            d[i][j] = [gx[1], gy[1], gz[1]] # we only compute for the central layer here

# this is the divergence of stress, it is force / volume
rx = ( d[0][0][0] + d[1][0][1] + d[2][0][2] )
ry = ( d[0][1][0] + d[1][1][1] + d[2][1][2] )
rz = ( d[0][2][0] + d[1][2][1] + d[2][2][2] )

res = np.zeros( (3, rx.shape[0], rx.shape[1]) )
res[0] = rx.copy()
res[1] = ry.copy()
res[2] = rz.copy()


In [ ]:
A = np.array([[ 1, 0, 0, 1, 1, 0 ],
              [ 0, 1, 0, 1, 0, 1 ],
              [ 0, 0, 1, 0, 1, 1 ]])
Ainv = np.linalg.pinv(A)
_, m, n = res.shape
dsig = (Ainv @ res.reshape( 3, m*n ) ).reshape(6, m,n)
print(Ainv)

In [ ]:

bins = np.arange(-61.5, 61.5, 4)
markers = ['^', 'o', 's', '*', 'd', '+']
components = [[0,0],[1,1],[2,2],[0,1],[0,2],[1,2]]

colors = np.array( [ [55,  126, 184],  [255, 127, 0]] )
colors = colors / np.max(colors, axis=1).reshape(2,1)
ci = [0,1,2,3,8,7]

for i,t in enumerate(titles):
    gb_mask = grain_boundaries[keys[1]]
    gb_mask = scipy.ndimage.binary_dilation(gb_mask, iterations=2)
    gbres = dsig[i,gb_mask]
    data = gbres[~np.isnan(gbres)].flatten() / 1e6
    print('$\Delta \sigma_{'+t+'}$ Grain Boundary', 'mean:', data.mean(), 'std:', data.std())
print('')
for i,t in enumerate(titles):
    mask = ~np.isnan(dsig[i,:,:])
    data = dsig[i,:,:][mask].flatten() / 1e6
    print('$\Delta \sigma_{'+t+'}$ Whole Layer   ', 'mean:', data.mean(), 'std:', data.std())


for i,t in enumerate(titles):

    gb_mask = grain_boundaries[keys[1]]
    gb_mask = scipy.ndimage.binary_dilation(gb_mask, iterations=2)

    gbres = dsig[i,gb_mask]

    data = gbres[~np.isnan(gbres)].flatten() / 1e6

    plt.figure(figsize=(7,7))
    h,b = np.histogram( data, bins=bins )
    h = h / np.sum(gb_mask)
    bc = (b[1:] + b[:-1])/2.
    plt.plot(bc, h, '--', color=colors[ci[0]], label='$\Delta \sigma_{'+t+'}$ Grain Boundary', marker='^', markersize=8)

    mask = ~np.isnan(dsig[i,:,:])
    data = dsig[i,:,:][mask].flatten() / 1e6
    h,b = np.histogram( data, bins=bins )
    h = h / np.sum(stress_mask)
    plt.plot(bc, h, '--', color=colors[ci[1]], label='$\Delta \sigma_{'+t+'}$ Whole Layer   ', marker='o', markersize=8)

    # variance increase with a factor of x2 at and close to the grain boundary

    plt.xlim([-60, 60])
    plt.ylim([-0.01, 0.28])
    plt.grid()
    #plt.legend()
    plt.xlabel('Out of Balance Stress $\Delta \sigma_{'+t+'}$ [MPa]')
    plt.ylabel('Volume Fraction [-]')
    if 1: plt.savefig('gallery/residual_stress_histogram_'+t+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
    plt.show()





In [ ]:
titles = 'xx yy zz xy xz yz'.split(' ')

for i,t in enumerate(titles):
    fig, ax = plt.subplots(1,1,figsize=(9,9))
    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)

    gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()
    
    rgb_arr = dsig[i][x_min:x_max+1, y_min:y_max+1].copy() / 1e6

    Xc = X[key][x_min:x_max+1, y_min:y_max+1]
    Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

    cmap = matplotlib.colormaps.get_cmap('RdBu_r')

    mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]
    
    vmax = 60
    vmin = -vmax 

    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

    rgb_image = cmap( norm(rgb_arr) )
    #rgb_image[gb, 0:3] = 0
    rgb_image[~mask, -1] = 0

    im = ax.pcolormesh( Xc, Yc, rgb_image )

    ax.set_title('$\sigma_{'+t+'}$')
    ax.axis('off')

    plt.savefig('gallery/dsig_'+t+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')

    plt.show()

In [ ]:
coords = np.argwhere(sample[keys[1]])
x_min, y_min = coords.min(axis=0)
x_max, y_max = coords.max(axis=0)

Xc = X[key][x_min:x_max+1, y_min:y_max+1]
Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

fig, ax = plt.subplots(1,1,figsize=(9,9))
ax.pcolormesh( Xc, Yc, gb_mask[x_min:x_max+1, y_min:y_max+1], cmap='gray_r' )
ax.axis('off')
plt.savefig('gallery/grain_boundary_mask_for_histograms_central_layer.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
plt.show()